## Test and Eval notebook

EECS MSc Project - Research Based - 2023 <br/><br/>
An Incremental Batch Learning Performance Assessment of CDS Implied Volatility n-step Estimation Approaches <br/><br/> Robert Taylor, Queen Mary, UoL

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install loguru

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
from loguru import logger
import contextlib
import tensorflow as tf
import matplotlib.pyplot as plt
from pandas.plotting import table
from contextlib import redirect_stdout, redirect_stderr
from sklearn.metrics import mean_absolute_error
from models import SVM, TFT_GRU, LGBM, Naive
from utils import ForecastStats, reconstruct_levels, rolling_test, plot_predictions_vs_actuals, plot_residuals, plot_residuals_violin

# set base dir to whatever
base_dir = '/content/drive/My Drive/Risk Forecasting'

sys.path.append(base_dir)

logger.remove() #low verbosity
logger.add(sys.stderr, level="ERROR")

# matplotlib in latex style
plt.rc('font', family='serif')
plt.rc('text', usetex=False)
plt.rc('mathtext', fontset='stix')

device_name = tf.test.gpu_device_name()
if device_name:
    print(f'{device_name} found')
else:
    print("No GPU found")

feature_matrix_path, target_vector_path = os.path.join(base_dir, 'features.pkl'), os.path.join(base_dir, 'RESPONSE Main 1m.xls')

feature_matrix = pd.read_pickle(feature_matrix_path)
target_vector = (pd.read_excel(target_vector_path, skiprows=range(6)).rename(columns=lambda x: x.strip()).assign(**{'Effective date': lambda df: pd
                .to_datetime(df['Effective date'], errors='coerce')}).dropna(subset=['Effective date']).set_index('Effective date'))

feature_matrix.index = pd.to_datetime(feature_matrix.index, errors='coerce')
target_vector.index = pd.to_datetime(target_vector.index, errors='coerce')

# Take ln for stationarity
target_vector = np.log(target_vector).diff().dropna()

# set sample range and take intersection
train_index = feature_matrix.index.intersection(target_vector.index)[:-42]

X = feature_matrix.loc[train_index]
y = target_vector.loc[train_index]

In [ ]:
X = X.iloc[:, :5] # n features

print(f"X shape: {X.shape}")

### test loop

In [ ]:
predictions = 42  # subset at end of sliced observations dedicated to estimates (model will be fit on observations - predictions)
sequence_length = 5  # length of sequences (set to 5 for trading week)
window = 42
log_path = os.path.join(base_dir, 'full_log.txt')

In [ ]:
with open(log_path, 'w') as f:
    with redirect_stdout(f), redirect_stderr(f):
        raw_metrics = {}
        levels_metrics = {}
        model_name_suffix = f' w {window}-period'

        # Naive model
        model_name = 'Naïve Forecast'
        raw_metrics[model_name] = rolling_test(
            Naive, X, y, chunk=(window + predictions), predictions=predictions, model_name=model_name
            )

        levels_metrics[model_name] = reconstruct_levels(target_vector, raw_metrics[model_name])

        # SVM model
        model_name = f'SVM{model_name_suffix}'
        with contextlib.redirect_stdout(sys.__stdout__):
            print(f'Training and rolling test for {model_name}')
        raw_metrics[model_name] = rolling_test(
            SVM, X, y, chunk=(window + predictions), predictions=predictions, sequence_length=sequence_length, model_name=model_name
            )

        levels_metrics[model_name] = reconstruct_levels(target_vector, raw_metrics[model_name])

        # LightGBM model
        model_name = f'LightGBM{model_name_suffix}'
        with contextlib.redirect_stdout(sys.__stdout__):
            print(f'Training and rolling test for {model_name}')
        raw_metrics[model_name] = rolling_test(
            LGBM, X, y, chunk=(window + predictions), predictions=predictions,sequence_length=sequence_length, model_name=model_name
            )

        levels_metrics[model_name] = reconstruct_levels(target_vector, raw_metrics[model_name])

        # TFT-GRU model
        model_name = f'TFT-GRU{model_name_suffix}'
        with contextlib.redirect_stdout(sys.__stdout__):
            print(f'Training and rolling test for {model_name}')
        with tf.device('/device:GPU:0'):
            raw_metrics[model_name] = rolling_test(
                TFT_GRU, X, y, chunk=(window + predictions), predictions=predictions,sequence_length=sequence_length, model_name=model_name
                )

        levels_metrics[model_name] = reconstruct_levels(target_vector, raw_metrics[model_name])

### eval

In [ ]:
# metrics, log returns
forecast_stats = ForecastStats(raw_metrics, X)
result_table = forecast_stats.calculate_forecast_stats()

result_table_path = os.path.join(base_dir, 'result_table.tex')
with open(result_table_path, 'w') as f:
    f.write(result_table.to_latex(index=False))

result_table

In [ ]:
# metrics, levels
level_metrics_table = forecast_stats.calculate_level_metrics(levels_metrics)

level_metrics_table_path = os.path.join(base_dir, 'level_metrics_table.tex')
with open(level_metrics_table_path, 'w') as f:
    f.write(level_metrics_table.to_latex(index=False))

level_metrics_table

### plots

In [ ]:
models = ['TFT-GRU', 'SVM', 'LightGBM']

In [ ]:
plot_predictions_vs_actuals(levels_metrics, models, model_name_suffix, window)

In [ ]:
plot_residuals(raw_metrics, models, model_name_suffix, window)

In [ ]:
all_residuals = []
for model in models:
    suffix = f'{model}{model_name_suffix}'
    residuals = np.array(raw_metrics[suffix]['actuals']) - np.array(raw_metrics[suffix]['forecasts'])
    all_residuals.append(residuals)

plot_residuals_violin(all_residuals, models, window)

In [ ]:
files = [os.path.join(base_dir, 'result_table2.png'), os.path.join(base_dir, 'level_metrics_table2.png')]

for df, file_name in zip([result_table, level_metrics_table], files):
    fig, ax = plt.subplots(figsize=(8, 3))
    ax.axis('tight')
    ax.axis('off')
    tbl = table(ax, df, loc='center', cellLoc='center', colWidths=[0.2]*len(df.columns))
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(12)
    tbl.scale(1.2, 1.2)
    plt.savefig(file_name, bbox_inches='tight')